In [1]:
# importar paquetes importantes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pandas_profiling

from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

In [2]:
df_train = pd.read_parquet("train.parquet")
df_test = pd.read_parquet("test.parquet")


In [ ]:
df_train.shape 

In [3]:
df_train.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,...,0,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn


In [ ]:
df_test.shape


In [ ]:
df_test

In [ ]:
train_profile = ProfileReport(df_train, title="Train", minimal=True)
test_profile = ProfileReport(df_test, title="Test")

In [ ]:
#test_profile


In [ ]:
#train_profile

In [3]:
#drop id, url, region url, laundry, parking, description, image url, lat, long
df_train['objetivo'] = df_train['price'].apply(lambda x: 1 if x < 999 else 0)
df_test['objetivo'] = None
df_train.drop(['id','url','state','region_url', 'laundry_options', 'parking_options', 'image_url','lat', 'long', 'description' ], axis= 1, inplace= True)
df_test.drop(['id','url','state','region_url', 'laundry_options', 'parking_options', 'image_url','lat', 'long', 'description' ], axis= 1, inplace= True)

In [4]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [6]:
columns_to_use = set(df_train.columns).intersection(set(df_test.columns))
columns_to_use







{'baths',
 'beds',
 'cats_allowed',
 'comes_furnished',
 'dogs_allowed',
 'electric_vehicle_charge',
 'region_SF bay area',
 'region_abilene',
 'region_akron / canton',
 'region_albany',
 'region_albuquerque',
 'region_altoona-johnstown',
 'region_amarillo',
 'region_ames',
 'region_anchorage / mat-su',
 'region_ann arbor',
 'region_annapolis',
 'region_appleton-oshkosh-FDL',
 'region_asheville',
 'region_ashtabula',
 'region_athens',
 'region_atlanta',
 'region_auburn',
 'region_augusta',
 'region_austin',
 'region_bakersfield',
 'region_baltimore',
 'region_baton rouge',
 'region_battle creek',
 'region_beaumont / port arthur',
 'region_bellingham',
 'region_bemidji',
 'region_bend',
 'region_billings',
 'region_binghamton',
 'region_birmingham',
 'region_bismarck',
 'region_bloomington',
 'region_bloomington-normal',
 'region_boise',
 'region_boone',
 'region_boston',
 'region_boulder',
 'region_bowling green',
 'region_bozeman',
 'region_brainerd',
 'region_brownsville',
 'region_b

In [7]:
X = df_train[columns_to_use]
y = df_train.objetivo

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)
clf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [11]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_train, y_train_pred))
print(metrics.accuracy_score(y_test, y_test_pred))

0.9809425346773498
0.9312395595553573


In [12]:

X_new = df_test[list(columns_to_use)]

y_pred = clf.predict(X_new)


In [16]:
df_y_pred = pd.DataFrame(y_pred,columns=["objetivo"])
df_y_pred.to_csv("Defo0.csv", index=False)